In [ ]:
import base64
import json
import os
import papermill as pm
import requests
import scrapbook as sb
import time

In [ ]:
# file_name
# base_name
# work_bucket

In [ ]:
data_path = os.getenv('DATA_PATH')
file_path = lambda *args: os.path.join(data_path, 'diygenomics-projects', 'experiment-a', work_bucket, *args)

endpoint = 'https://api.mathpix.com/v3/'
extensions = ['.md', '.mmd', '.docx', '.tex', '.html', '.lines.json', '.lines.mmd.json']
app_id = os.getenv('MATHPIX_APP_ID')
app_key = os.getenv('MATHPIX_APP_KEY')

headers = {
    'app_id': app_id,
    'app_key': app_key,
}

conversion_options = {
    'conversion_formats': {'docx': True, 'tex.zip': True},
    'math_inline_delimiters': ['$', '$'],
    'rm_spaces': True
}

In [ ]:
def download_file(pdf_id, extension):
    file_with_extension = f'{pdf_id}{extension}'
    url = os.path.join(endpoint, 'pdf', file_with_extension) 
    response = requests.get(url, headers=headers)
    if extension == '.tex':
        file_with_extension = f'{file_with_extension}.zip'
        
    if extension == '.mmd':
        with open(file_path(base_name, 'mathpix', file_with_extension), "w") as f:
                f.write(response.text) 
    else:
         with open(file_path(base_name, 'mathpix', file_with_extension), "wb") as f:
                f.write(response.content)        

In [ ]:
pdf_id = None

while True:
    r = requests.post(os.path.join(endpoint, 'pdf'),
        headers=headers,
        data={
            'options_json': json.dumps(conversion_options)
        },
        files={
            'file': open(file_path(file_name), 'rb')
        }
    )
    
    try:
        json_data = r.json()
        if 'pdf_id' in json_data:
            pdf_id = json_data['pdf_id']
            break
    except ValueError:
        print(f'Invalid JSON response.\n{r}')
    time.sleep(60)

In [ ]:
print(pdf_id)

In [ ]:
status = None

while True:
    r = requests.get(os.path.join(endpoint, 'converter', pdf_id), headers=headers)
    try:
        json_data = r.json()
        if 'status' in json_data:
            status = json_data['status']
            if status == 'completed' or status == 'error':
                break
    except ValueError:
        print(f'Invalid JSON response. {r}')
    time.sleep(60)

In [ ]:
if status == 'completed':
    for extension in extensions:
        download_file(pdf_id, extension)

In [ ]:
sb.glue('external_id', pdf_id)
sb.glue('status', status)

In [ ]:
# r = requests.get(os.path.join(endpoint, 'pdf', pdf_id), headers=headers)